<a href="https://colab.research.google.com/github/dylankenny/Lab-1/blob/main/Assignment_2_Learning_Bayesian_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The purpose of this notebook is to gain some practice in learning graphical models. Your goal is to:

1.   load the Breast Cancer (categorical) data set: https://archive.ics.uci.edu/ml/datasets/breast+cancer
2.   keep the last 20% of the data for testing
3.   compare the performance of 3 learned models on the test data: naive Bayes, tree-structured BN (using the Chow-Liu algorithm), and BN

Below I provide some code fragments to assist with this task.

**Marks:**
*   70%: successful learning of 3 models
*   30%: critical discussion of the reasons for any differences in predictive accuracy



In [31]:
!pip install  pgmpy
!pip install pandas
!pip install numpy
!pip install ucimlrepo

In [70]:
import numpy as np
import pandas as pd
import missingno as mno
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.impute import SimpleImputer
from sklearn import metrics
from pgmpy.estimators import MaximumLikelihoodEstimator, TreeSearch, BayesianEstimator, PC, HillClimbSearch, ExhaustiveSearch
from pgmpy.estimators import BDeuScore, K2Score, BicScore
from pgmpy.models import BayesianModel, BayesianNetwork, NaiveBayes
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [33]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
breast_cancer = fetch_ucirepo(id=14)

# data (as pandas dataframes)
X = breast_cancer.data.features
y = breast_cancer.data.targets

# metadata
print(breast_cancer.metadata)

# variable information
print(breast_cancer.variables)


{'uci_id': 14, 'name': 'Breast Cancer', 'repository_url': 'https://archive.ics.uci.edu/dataset/14/breast+cancer', 'data_url': 'https://archive.ics.uci.edu/static/public/14/data.csv', 'abstract': 'This breast cancer domain was obtained from the University Medical Centre, Institute of Oncology, Ljubljana, Yugoslavia. This is one of three domains provided by the Oncology Institute that has repeatedly appeared in the machine learning literature. (See also lymphography and primary-tumor.)', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 286, 'num_features': 9, 'feature_types': ['Categorical'], 'demographics': ['Age'], 'target_col': ['Class'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1988, 'last_updated': 'Thu Mar 07 2024', 'dataset_doi': '10.24432/C51P4M', 'creators': ['Matjaz Zwitter', 'Milan Soklic'], 'intro_paper': None, 'additional_info': {'summary': 'Thi

In [34]:
nan_count1 = X.isna().sum()
print(nan_count1)

age            0
menopause      0
tumor-size     0
inv-nodes      0
node-caps      8
deg-malig      0
breast         0
breast-quad    1
irradiat       0
dtype: int64


In [50]:
data = pd.concat([X, y], axis=1)

In [64]:
train = data.sample(frac = 0.8, random_state = 1)
test = data.drop(train.index)


# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) # Added random_state for reproducibility
# Assume 'X' is your DataFrame and 'class' is the target column
X = data.drop(columns=['Class'])
y = data['Class']


In [65]:
# Create a LabelEncoder instance
le = LabelEncoder()

# Iterate through all columns in X_train and X_test
for column in X_train.columns:
    # If the column contains string values, apply Label Encoding
    if pd.api.types.is_string_dtype(X_train[column]):
        # Fit the LabelEncoder on the combined unique values from both datasets
        le.fit(pd.concat([X_train[column], X_test[column]]).astype(str).unique())
        # Transform both training and testing data
        X_train[column] = le.transform(X_train[column].astype(str))
        X_test[column] = le.transform(X_test[column].astype(str))

First learn a naive Bayes model

In [86]:
from sklearn.naive_bayes import GaussianNB
# create the structure manually to create model_struct

# from pgmpy.estimators import MaximumLikelihoodEstimator

# mle = MaximumLikelihoodEstimator(model=model_struct, data= ****)

model1 = GaussianNB()
model1.fit(X_train, y_train)

model1_pred = model1.predict(X_test)
model1_acc = accuracy_score(y_test, model1_pred)
print(f'Naive Bayes Accuracy {model1_acc}')

Naive Bayes Accuracy 0.7209302325581395


Next, learn a tree-structured model

In [68]:
from pgmpy.estimators import TreeSearch

# learn graph structure
est = TreeSearch(train, root_node="age")
dag = est.estimate(estimator_type="chow-liu")

from pgmpy.estimators import BayesianEstimator

# there are many choices of parametrization, here is one example
model2 = BayesianNetwork(dag.edges())
model2.fit(
    train, estimator=BayesianEstimator, prior_type="dirichlet", pseudo_counts=0.1
)
model2.get_cpds()

Building tree:   0%|          | 0/45.0 [00:00<?, ?it/s]

[<TabularCPD representing P(age:6) at 0x7e18b4ebb580>,
 <TabularCPD representing P(menopause:3 | age:6) at 0x7e18b4ebb3a0>,
 <TabularCPD representing P(tumor-size:11 | age:6) at 0x7e18b4ebb250>,
 <TabularCPD representing P(inv-nodes:7 | tumor-size:11) at 0x7e18b4ebad40>,
 <TabularCPD representing P(breast-quad:5 | tumor-size:11) at 0x7e18b4ef4940>,
 <TabularCPD representing P(deg-malig:3 | tumor-size:11) at 0x7e18b4ef4160>,
 <TabularCPD representing P(node-caps:2 | inv-nodes:7) at 0x7e18b4ef4880>,
 <TabularCPD representing P(irradiat:2 | inv-nodes:7) at 0x7e18b4ef4100>,
 <TabularCPD representing P(breast:2 | breast-quad:5) at 0x7e18b4ef4790>,
 <TabularCPD representing P(Class:2 | deg-malig:3) at 0x7e18b4ef4130>]

In [85]:
model2_pred = model2.predict(X_test)
model2_acc = accuracy_score(y_test, model2_pred)
print(f'TreeSearch Accuracy: {model2_acc}')

  0%|          | 0/83 [00:00<?, ?it/s]

TreeSearch Accuracy: 0.6976744186046512


Finally learn a Bayesian network. **First learn the structure, and then the parameters.**

# **Learning Bayesian Networks**

We now want to learn a Bayesian network, given a set of sample data. Learning a Bayesian network can be split into two problems:

 **Structure learning**: Given a set of data samples, estimate a DAG that captures the dependencies between the variables.

  **Parameter learning**: Given a set of data samples and a DAG that captures the dependencies between the variables, estimate the (conditional) probability distributions of the individual variables.


Methods for doing this include:

Structure learning for discrete, fully observed networks:
    
*    Score-based structure estimation (BIC/BDeu/K2 score; exhaustive search, hill climb/tabu search)
*   Constraint-based structure estimation (PC)

Parameter learning for discrete nodes:

*   Maximum Likelihood Estimation
*   Bayesian Estimation
    





**Structure Learning**

You can use MLE or Bayesian estimation methods.

MLE State counts

To make sense of the given data, we can start by counting how often each state of the variable occurs. If the variable is dependent on parents, the counts are done conditionally on the parents states, i.e. for separately for each parent configuration:

**Bayesian Parameter Estimation**


The Bayesian Parameter Estimator starts with already existing prior CPDs, that express our beliefs about the variables before the data was observed. Those "priors" are then updated, using the state counts from the observed data.

One can think of the priors as consisting in pseudo state counts, that are added to the actual counts before normalization. Unless one wants to encode specific beliefs about the distributions of the variables, one commonly chooses uniform priors, i.e. ones that deem all states equiprobable.

A very simple prior is the so-called K2 prior, which simply adds 1 to the count of every single state. A somewhat more sensible choice of prior is BDeu (Bayesian Dirichlet equivalent uniform prior). For BDeu we need to specify an equivalent sample size N and then the pseudo-counts are the equivalent of having observed N uniform samples of each variable (and each parent configuration).

**Maximum Likelihood Estimation**


A natural estimate for the CPDs is to simply use the relative frequencies, with which the variable states have occured.

This approach is Maximum Likelihood Estimation (MLE). According to MLE, we should fill the CPDs in such a way, that $P(\text{data}|\text{model})$ is maximal. This is achieved when using the relative frequencies.  pgmpy supports MLE as follows:


mle.estimate_cpd(variable) computes the state counts and divides each cell by the (conditional) sample size. The mle.get_parameters()-method returns a list of CPDs for all variable of the model.

The built-in fit()-method of BayesianModel provides more convenient access to parameter estimators:

# **Structure Learning**




To learn model structure (a DAG) from a data set, there are two broad techniques:

*   score-based structure learning
*   constraint-based structure learning

In this assignment focus on the score-based approach.


# **Score-based Structure Learning**


This approach construes model selection as an optimization task. It has two building blocks:

A scoring function $s_D\colon M \to \mathbb R$ that maps models to a numerical score, based on how well they fit to a given data set $D$.
A search strategy to traverse the search space of possible models $M$ and select a model with optimal score.


**Scoring functions**


Commonly used scores to measure the fit between model and data are Bayesian Dirichlet scores such as BDeu or K2 and the Bayesian Information Criterion (BIC, also called MDL).


**Search strategies**


The search space of DAGs is super-exponential in the number of variables and the above scoring functions allow for local maxima. The first property makes exhaustive search intractable for all but very small networks, the second prohibits efficient local optimization algorithms to always find the optimal structure. Thus, identifiying the ideal structure is often not tractable. Despite these bad news, heuristic search strategies often yields good results.


Heuristic search: HillClimbSearch implements a greedy local search that starts from the DAG start (default: disconnected DAG) and proceeds by iteratively performing single-edge manipulations that maximally increase the score. The search terminates once a local maximum is found.


The estimated values in the CPDs are now more conservative.

BayesianEstimator, too, can be used via the fit()-method.

In [84]:
est = HillClimbSearch(data)
best_model = est.estimate(scoring_method=K2Score(train))

# Define all features manually
all_features = ['age', 'menopause', 'tumor-size', 'inv-nodes', 'node-caps',
                'deg-malig', 'breast', 'breast-quad', 'irradiat']

# Create the Bayesian Network with all features
BN_model = BayesianNetwork(best_model.edges())  # Use BayesianNetwork
BN_model.add_nodes_from(all_features)  # Add any missing features

BN_model.fit(train, estimator=MaximumLikelihoodEstimator)

# Now predict using the original X_test
bn_pred = BN_model.predict(X_test)

bn_acc = accuracy_score(y_test, bn_pred)
print(f'Bayesian Network Accuracy: {bn_acc}')

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/83 [00:00<?, ?it/s]

Bayesian Network Accuracy: 0.9186046511627907





# **Discussion**

Please critically compare the performance of the 3 different models.